In [1]:
import serial
import time

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
PICO_PORT = '/dev/ttyACM0'

MHZ = 1000000

In [2]:
# helper for sending serial commands
# expects a string with the command (they dont have to be escaped with \r or \n at the end)
# if echo is set to false, that means not to worry about the response from the
# pico it signifigantly reduces communication time when sending many
# instructions, but you lose out on the debugging info from the pico
def send(command: str, echo = True) -> str:
    # pico is expecting a newline to end every command
    if command[-1] != '\n':
        command += '\n'

    resp = ''
    conn = None
    try:
        conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)
        conn.write(command.encode())
        if echo:
            resp = conn.readlines()
            resp = "".join([s.decode() for s in resp])

    except Exception as e:
        print("Encountered Error: ", e)

    finally:
        conn.close()

    return resp


## Test Serial Communication with the Pico

In [9]:
assert send('reset')    == 'ok\r\n'
assert send('status')   == '0\r\n'
assert send('version')  == '0.1.1\r\n'
print('Serial Communication Successful')

Serial Communication Successful


## Test Register Readback

In [10]:
    # Helper for reading register values and putting them in a dictionary
    # takes in the frequency of the reference clock, assuming default of 125 MHz
    def readregs(ref_clk = 125 * MHZ) -> dict:
        ad9959 = {}

        regs = send('readregs')
        regs = regs.split('\r\n')
        
        # strip out register labels
        regs = [''.join(r.split()[1:]) for r in regs]

        # convert from hex to decimal
        for i, reg in enumerate(regs):
            try:
                regs[i] = int(reg, 16)
            except ValueError:
                pass

        # mask and shift to pull out PLL Multiplier from fr1
        ad9959['pll_mult'] = (regs[1] & 0x7c0000) >> 18

        # will need system clock to find the frequencies from the tuning words
        sys_clk = ref_clk * ad9959['pll_mult']
        #print(sys_clk)
        for i in range(4):
            ad9959[i] = {}

            ftw = regs[5 + 9 * i]
            ad9959[i]['freq'] = ftw / 2**32 * sys_clk

            pow = regs[6 + 9 * i]
            ad9959[i]['phase'] = pow * 360 / 2**14

            acr = regs[7 + 9 * i]
            if acr & 0x001000:
                ad9959[i]['amp'] = (acr & 0x0003ff) / 1023
            else:
                ad9959[i]['amp'] = 1

        return ad9959

    readregs()


{'pll_mult': 0,
 0: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 1: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 2: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 3: {'freq': 0.0, 'phase': 0.0, 'amp': 1}}

In [5]:
assert send('reset') == 'ok\r\n', 'Could not run "reset" command'
ad9959 = readregs()
for i in range(4):
    assert ad9959[i]['freq'] == 0
    assert ad9959[i]['phase'] == 0
    assert ad9959[i]['amp'] == 1

send('setamp 2 0.5')
send('setfreq 0 100000000')
send('setphase 1 270')

ad9959 = readregs()

#print(ad9959[2]['amp'])

assert abs(ad9959[2]['amp'] - 0.5) < 0.01
assert abs(ad9959[0]['freq'] - 100 * MHZ) < 1
assert abs(ad9959[1]['phase'] - 270) < 1

print('Register Readback Successful')


AssertionError: 

## Test Single Stepping Table Mode

Program a 2000 step table that single steps from 10 MHz to 100 MHz over the corse of 2 seconds.
The resulting sweep can easily be seen with a spectrum analyzer. 
It is then automatically executed and checks that all 2000 triggers were processed successfully.

In [6]:
startPoint = 10 * MHZ
endPoint = 100 * MHZ
totalTime = 0.01 # sec

spacing = 1000 * 10**(-6) # us
steps = round(totalTime / spacing)
delta = (endPoint - startPoint) / steps

send('debug off')
send('mode 0 1')
send('setchannels 1')

for i in range(steps):
    send(f'set 0 {i} {startPoint + delta * i} 1 0 {spacing * 10**9 / 8}', echo=False)
assert send(f'set 4 {i + 1}') == "ok\r\n"

print("Table Programmed, Executing")
assert send('start') == 'ok\r\n', 'Buffered Execution did not start correctly'
time.sleep(totalTime)
assert send('numtriggers') == f'{steps}\r\n', 'Wrong number of triggers processed'
print('Table Executed successfully')


AssertionError: 

### Test Non-Volatile Storage
Run this test after the previous one to test storing and retrieving table instructions in non-volatile memory that will survive a power cycle.

In [9]:
send('save')

# # destory current table
time.sleep(2)
for i in range(steps):
    send(f'set 0 {i} 0 0 0 0', echo=False)

# load and run table
send('load')
time.sleep(1)
send('mode 0 1')
send('setchannels 1')
send('start')

time.sleep(2)
assert send('numtriggers') == '2000\r\n', 'Something went wrong'
print('Table run from non-volatile memory successfully')


AssertionError: Something went wrong

## Amplitude Sweep

### Pico Start

In [10]:
send('debug off')
send("""mode 1 1
setchannels 1
setfreq 0 100000000
setfreq 1 100000000
setfreq 2 100000000
setfreq 3 100000000
set 0 0 1.0 0.0 0.001 1 2000
set 0 1 0.0 0.5 0.001 1 2000
set 0 2 0.5 1.0 0.001 1 2000
set 0 3 1.0 0.0 0.001 1 2000
set 0 4 0.0 1.0 0.001 1 2000
set 0 5 1.0 0.5 0.001 1 2000
set 0 6 0.5 0.0 0.001 1 2000
set 0 7 0.0 1.0 0.001 1 2000
set 4 8
start
""")

assert send('numtriggers') == '8\r\n'
print('Success')

Success


### HWStart

In [11]:
send('debug off')
send("""abort
mode 1 1
setchannels 1
setfreq 0 100000000
setfreq 1 100000000
setfreq 2 100000000
setfreq 3 100000000
set 0 0 1.0 0.0 0.001 1 2000
set 0 1 0.0 0.5 0.001 1 2000
set 0 2 0.5 1.0 0.001 1 2000
set 0 3 1.0 0.0 0.001 1 2000
set 0 4 0.0 1.0 0.001 1 2000
set 0 5 1.0 0.5 0.001 1 2000
set 0 6 0.5 0.0 0.001 1 2000
set 0 7 0.0 1.0 0.001 1 2000
set 4 8
hwstart
""")

'ok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\nok\r\n'

This produces the following scope trace:  
<img src="amp-test.png" alt="Amplitude Sweep Test on Oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is any of the 4 channel outputs from the AD9959

## Frequency Sweep

In [64]:
f112 = 112e6
f115 = 115e6
f118 = 118e6

d = 2000
t = 3000

send(
f"""abort
mode 2 1
setchannels 1
set 0 0 {f112} {f118} {d}   1 {t}
set 0 1 {f118} {f115} {d/4} 1 {t*2}
set 0 2 {f115} {f112} {d}   1 {t/2}
set 0 3 {f112} {f112} {0}   1 {t}
set 0 4 {f115} {f115} {0}   1 {t}
set 0 5 {f118} {f118} {0}   1 {t}
set 0 6 {f118} {f112} {d}   1 {t}

set 0 7 {f112} {f115} {d/4} 1 {t*2}
set 0 8 {f115} {f118} {d}   1 {t*10}
set 0 9 {f112} {f112}  0    1 1
set 4 10
start
"""
)

assert send('numtriggers') == '10\r\n'
print('Success')

Success


Produces the following scope trace:  
<img src="freq-test.png" alt="Frequency Sweep Example on oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is any of the 4 channel outputs from the AD9959  
The green trace is the output from an interferometer



The following code is for the same trace, but with an external trigger:

In [65]:
send(
f"""abort
mode 2 0
setchannels 1
set 0 0 {f112} {f118} {d} 1 {t}
set 0 1 {f118} {f115} {d} 1 {t}
set 0 2 {f115} {f112} {d} 1 {t}
set 0 3 {f112} {f115} {d} 1 {t}
set 0 4 {f115} {f118} {d} 1 {t}
set 0 5 {f118} {f112} {d} 1 {t}
set 5 6
hwstart
"""
)

'ok\r\nok\r\nok\r\nSet ins #0 for channel 0 from 111999999.964610 Hz to 118000000.016764 Hz with delta 2000.015229 Hz and rate of 1\r\nok\r\nSet ins #1 for channel 0 from 118000000.016764 Hz to 114999999.990687 Hz with delta 2000.015229 Hz and rate of 1\r\nok\r\nSet ins #2 for channel 0 from 114999999.990687 Hz to 111999999.964610 Hz with delta 2000.015229 Hz and rate of 1\r\nok\r\nSet ins #3 for channel 0 from 111999999.964610 Hz to 114999999.990687 Hz with delta 2000.015229 Hz and rate of 1\r\nok\r\nSet ins #4 for channel 0 from 114999999.990687 Hz to 118000000.016764 Hz with delta 2000.015229 Hz and rate of 1\r\nok\r\nSet ins #5 for channel 0 from 118000000.016764 Hz to 111999999.964610 Hz with delta 2000.015229 Hz and rate of 1\r\nok\r\nok\r\nok\r\n'

## Phase Sweep

In [66]:
t = 2000
d = 0.2

send(f"""abort
debug off
setfreq 0 100000000
setfreq 1 100000000
setfreq 2 100000000
setfreq 3 100000000
setphase 0 0
setphase 1 0
setphase 2 0
setphase 3 0
mode 3 1
setchannels 2
set 0 0 0 0 0 0 {t}
set 0 1 0 0 0 0 {t}
set 0 2 0 0 0 0 {t}
set 0 3 0 0 0 0 {t}
set 0 4 0 0 0 0 {t}
set 0 5 0 0 0 0 {t}
set 1 0 0 180 {d} 1 {t}
set 1 1 180 90 {d} 1 {t}
set 1 2 90 0 {d} 1 {t}
set 1 3 0 90 {d} 1 {t}
set 1 4 90 180 {d} 1 {t}
set 1 5 180 0 {d} 1 {t}
set 4 6
start
""")

assert send('numtriggers') == '6\r\n'
print('Success')

Success


This produces the following scope trace:    
<img src="phase-test.png" alt="Phase Sweep Test on Oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is channel 1 from the AD9959  
The pink trace is the output of a phase frequency detector between channels 0 and 1 of the AD9959

## Frequency Sweep and Single Step Other Params

In [67]:
f112 = 112e6
f115 = 115e6
f118 = 118e6

d = 20000
t = 30000

send(
f"""abort
mode 5 1
setchannels 2
set 0 0 {f112} {f118} {d}   1 1 0 {t}
set 0 1 {f118} {f115} {d/4} 1 1 0 {t*2}
set 0 2 {f115} {f112} {d}   1 1 0 {t/2}
set 0 3 {f112} {f112} {0}   1 1 0 {t}
set 0 4 {f115} {f115} {0}   1 1 0 {t}
set 0 5 {f118} {f118} {0}   1 1 0 {t}
set 0 6 {f118} {f112} {d}   1 1 0 {t}
set 0 7 {f112} {f115} {d/4} 1 1 0 {t*2}
set 0 8 {f115} {f118} {d}   1 1 0 {t*10}
set 0 9 {f112} {f112}  0    1 1 0 1
set 1 0 {f112} {f118} {d}   1 0.75 90 {t}
set 1 1 {f118} {f115} {d/4} 1 0.5  180 {t*2}
set 1 2 {f115} {f112} {d}   1 0.9  90 {t/2}
set 1 3 {f112} {f112} {0}   1 1.0  00 {t}
set 1 4 {f115} {f115} {0}   1 0.9  90 {t}
set 1 5 {f118} {f118} {0}   1 0.8  180 {t}
set 1 6 {f118} {f112} {d}   1 0.9  90 {t}
set 1 7 {f112} {f115} {d/4} 1 1.0  00 {t*2}
set 1 8 {f115} {f118} {d}   1 0.9  90 {t*10}
set 1 9 {f112} {f112}  0    1 1.0  90 1
set 4 10
start
"""
)

assert send('numtriggers') == '10\r\n'
print('Success')

Success


This produces the following scope trace:    
<img src="all_sweep.png" alt="Phase Sweep Test on Oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is channel 1 from the AD9959  
The pink trace is the output of a phase frequency detector between channels 0 and 1 of the AD9959
The green trace is the interferometer

In [18]:
send('setfreq1')

'set freq: 85500000.044703\r\nok\r\n'

In [21]:
readregs()

{'pll_mult': 4,
 0: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 1: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 2: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 3: {'freq': 0.0, 'phase': 0.0, 'amp': 1}}

In [24]:
send('pattern2')

''

In [8]:
send('checkv')

''

In [23]:
send('pattern8')

''

In [25]:
send('reset')

''